This notebook needs tests naive ML models for disease classification using the current vessel features for the left eye only. 

*Further improvements to be made !*

# Imports

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Datasets Preparation and Preprocessing

In [34]:
# load the left dataset
df_left = pd.read_csv(r'C:\Users\ioana\Documents\VS Projects\Quantitative-Big-Immaging-Project\Datasets\OIA-ODIR-pre-2\vessel_features_cleaned_left.csv')
# load the right dataset
df_right = pd.read_csv(r'C:\Users\ioana\Documents\VS Projects\Quantitative-Big-Immaging-Project\Datasets\OIA-ODIR-pre-2\vessel_features_cleaned_right.csv')

In [35]:
df_left

,dataset,Image,vessel_density,dens_0_0,dens_0_1,dens_1_0,dens_1_1,fractal_dim,branch_angle,mean_width,...,Patient_Age,Patient Sex,N,D,G,C,A,H,M,O
0,Training Set,0_left,0.018013,0.013840,0.012436,0.015808,0.029968,1.488414,95.738045,1.133849,...,69,Female,0,0,0,1,0,0,0,0
1,Training Set,1005_left,0.058506,0.086105,0.048782,0.062546,0.036591,1.589583,99.866129,1.491264,...,78,Female,0,0,0,0,0,0,0,1
2,Training Set,1006_left,0.046089,0.057037,0.032944,0.068329,0.026047,1.559135,99.229892,1.362923,...,64,Male,0,0,0,0,0,0,0,1
3,Training Set,1008_left,0.076565,0.091705,0.071213,0.081772,0.061569,1.643733,99.953220,1.491798,...,65,Male,0,0,0,0,0,0,0,1
4,Training Set,100_left,0.085808,0.109344,0.072128,0.093338,0.068420,1.674093,100.025915,1.538602,...,59,Male,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Off-site Test Set,4794_left,0.043018,0.061874,0.038727,0.039856,0.031616,1.504387,100.235208,1.737682,...,62,Male,0,1,0,0,0,0,0,0
4996,Off-site Test Set,937_left,0.049614,0.068817,0.035782,0.057739,0.036118,1.580805,99.404827,1.380281,...,60,Female,0,1,0,0,0,1,0,0
4997,Off-site Test Set,967_left,0.097431,0.114166,0.097031,0.101196,0.077332,1.691917,100.103683,1.562734,...,21,Female,0,1,0,0,0,0,0,1
4998,Off-site Test Set,988_left,0.055634,0.080963,0.044785,0.063141,0.033646,1.555367,99.907835,1.522229,...,60,Female,0,1,0,0,0,0,0,1


In [12]:
# pick the row that have the dataset column equal to 'Training Set'
df_left.drop(columns=['Patient Sex'], inplace=True)
training_set_left = df_left[df_left['dataset'] == 'Training Set']
training_set_left = training_set_left.drop(columns=['dataset'])
# the rest is the test set
test_set_left = df_left[df_left['dataset'] != 'Training Set']
test_set_left = test_set_left.drop(columns=['dataset'])

In [14]:
test_set_left.columns

Index(['Image', 'vessel_density', 'dens_0_0', 'dens_0_1', 'dens_1_0',
       'dens_1_1', 'fractal_dim', 'branch_angle', 'mean_width', 'std_width',
       'ID', 'Patient_Age', 'N', 'D', 'G', 'C', 'A', 'H', 'M', 'O'],
      dtype='object')

# Train-Test Spliting - Left

In [24]:
feature_cols = [
    'vessel_density', 'dens_0_0', 'dens_0_1', 'dens_1_0', 'dens_1_1',
    'fractal_dim', 'branch_angle', 'mean_width', 'std_width', 'Patient_Age'
]

label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

X_train = training_set_left[feature_cols]
Y_train = training_set_left[label_cols]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
#X_train_scaled = X_train

X_test = test_set_left[feature_cols]
Y_test = test_set_left[label_cols]
X_test_scaled = scaler.transform(X_test) 
#X_test_scaled = X_tesy

# KNN

In [25]:
# Grid of neighbor values to try
param_grid = {
    'estimator__n_neighbors': [5, 7, 9, 11,13, 15, 17, 19, 21],
    'estimator__weights': ['uniform', 'distance'],
    'estimator__metric': ['euclidean', 'manhattan', 'minkowski'],
    'estimator__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}

knn = KNeighborsClassifier()
clf = OneVsRestClassifier(knn)

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1
)

grid_search.fit(X_train_scaled, Y_train)

# Best model and test evaluation
best_model = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)


Best hyperparameters: {'estimator__algorithm': 'auto', 'estimator__metric': 'manhattan', 'estimator__n_neighbors': 5, 'estimator__weights': 'uniform'}


In [26]:
Y_pred = best_model.predict(X_test_scaled)
print(classification_report(Y_test, Y_pred, target_names=label_cols))

              precision    recall  f1-score   support

           N       0.34      0.22      0.26       486
           D       0.39      0.24      0.30       490
           G       0.08      0.02      0.04        90
           C       0.63      0.44      0.52        96
           A       0.00      0.00      0.00        74
           H       0.00      0.00      0.00        46
           M       0.39      0.13      0.20        69
           O       0.24      0.13      0.17       411

   micro avg       0.35      0.19      0.24      1762
   macro avg       0.26      0.15      0.19      1762
weighted avg       0.31      0.19      0.23      1762
 samples avg       0.21      0.19      0.20      1762



c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Random Forest

In [27]:
rf = RandomForestClassifier(random_state=42)
clf = OneVsRestClassifier(rf)

param_grid = {
    'estimator__n_estimators': [100, 200],
    'estimator__max_depth': [None, 10, 20],
    'estimator__min_samples_split': [2, 5],
    'estimator__min_samples_leaf': [1, 2],
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1
)

grid_search.fit(X_train_scaled, Y_train)
best_model = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)

Best hyperparameters: {'estimator__max_depth': None, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 100}


In [28]:
Y_test_pred = best_model.predict(X_test_scaled)
print("\nTest set performance:")
print(classification_report(Y_test, Y_test_pred, target_names=label_cols))


Test set performance:
              precision    recall  f1-score   support

           N       0.38      0.11      0.17       486
           D       0.44      0.16      0.23       490
           G       0.00      0.00      0.00        90
           C       0.65      0.49      0.56        96
           A       0.00      0.00      0.00        74
           H       0.00      0.00      0.00        46
           M       0.50      0.12      0.19        69
           O       0.23      0.03      0.06       411

   micro avg       0.43      0.11      0.18      1762
   macro avg       0.28      0.11      0.15      1762
weighted avg       0.34      0.11      0.16      1762
 samples avg       0.13      0.12      0.12      1762



c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# XGBoost

In [29]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
clf = OneVsRestClassifier(xgb)

param_grid = {
    'estimator__n_estimators': [100, 200, 300],
    'estimator__max_depth': [3, 5, 7, 9],
    'estimator__learning_rate': [0.05, 0.1, 0.01],
    'estimator__subsample': [0.8, 1.0],
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1
)

grid_search.fit(X_train_scaled, Y_train)

best_model = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)

c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:19:17] WARNING: D:\bld\xgboost-split_1745056707810\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:19:18] WARNING: D:\bld\xgboost-split_1745056707810\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:19:19] WARNING: D:\bld\xgboost-split_1745056707810\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best hyperparameters: {'estimator__learning_rate': 0.1, 'estimator__max_depth': 5, 'estimator__n_estimators': 300, 'estimator__subsample': 0.8}


In [30]:
Y_test_pred = best_model.predict(X_test_scaled)
print("\nTest set performance:")
print(classification_report(Y_test, Y_test_pred, target_names=label_cols))


Test set performance:
              precision    recall  f1-score   support

           N       0.37      0.20      0.26       486
           D       0.40      0.21      0.28       490
           G       0.06      0.02      0.03        90
           C       0.64      0.49      0.55        96
           A       0.00      0.00      0.00        74
           H       0.00      0.00      0.00        46
           M       0.34      0.16      0.22        69
           O       0.26      0.09      0.14       411

   micro avg       0.37      0.17      0.23      1762
   macro avg       0.26      0.15      0.18      1762
weighted avg       0.33      0.17      0.22      1762
 samples avg       0.19      0.18      0.18      1762



c:\Users\ioana\anaconda3\envs\qbi_env_2\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
assert Y_test.shape == Y_test_pred.shape

# Plot a confusion matrix for each class
for i, label in enumerate(label_cols):
    cm = confusion_matrix(Y_test.iloc[:, i], Y_test_pred[:, i])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f'Not {label}', label])
    
    plt.figure(figsize=(3, 3))
    disp.plot(cmap='Blues', values_format='d')
    plt.title(f'Confusion Matrix for {label}')
    plt.tight_layout()
    plt.show()